In [323]:
#-------------------------------------------------------Library imports----------------------------------------------------------
#Importo la liberia numpy perchè è un pacchetto che fornisce oggetti di tipo array multidimensionali, operazioni veloci sui array etc
import numpy as np
#Importo la libreria di panda perchè : esistono metodi build-in in python che si sovrapongono con i metodi di panda.
#Es map() / all() / any() / filter() etc. Per evitare le collisioni di questi metodi importo la libreria di panda
import pandas as pd
#Importo la libreria seaborn per poter visualizzare i drawing 
import seaborn as sns
#Importo la libreria di pickle per poter serializzare e deserializzare le 
#strutture di oggetti Python chiamate anche marshalling o appiattimento.
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn import model_selection 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline
#-------------------------------------------------------Dataset import----------------------------------------------------------
import_dataset = 'datasets/ATP_dataset_2019_2011.xlsx'
imd = pd.read_excel(import_dataset) 

#imd - > import dataset

In [324]:
#in questo punto tolgo tutte le collone che secondo me non servono per il mio lavoro
imd = imd.drop(['ATP','Tournament','Date','Series','B365W','B365L','EXW','EXL','LBW','LBL','PSW','PSL','SJW','SJL','MaxW','MaxL','AvgW','AvgL','LPts','W1','L1','W2','L2','W3','L3','W4','L4','W5','L5','Wsets','Lsets','Comment','WPts','Best of','Location','Court'], axis=1)


In [325]:
imd.index.name   = "Match ID"  #il mio index da adesso in poi si chiamerà Match ID 
imd.columns.name = "Attributes" #la riga degli attributi tipo surface / round verra chiamata attributi

In [326]:
# A questo punto importo il dataset con i player qualificati ad AustralianOpen 2019.
# Il dataset lo creato io visto che non c'era già uno creato online
import_dataset2 = 'datasets/ATP_AO_QP.xlsx'
imd2 = pd.read_excel(import_dataset2)
x = {i: p for i, p in enumerate(imd2['Player'].unique())}
x

{0: 'Djokovic N.',
 1: 'Nadal R.',
 2: 'Federer R.',
 3: 'Zverev A.',
 4: 'Anderson K.',
 5: 'Čilić M.',
 6: 'Thiem D.',
 7: 'Nishikori K.',
 8: 'Isner J.',
 9: 'Khachanov K.',
 10: 'Ćorić B.',
 11: 'Fognini F.',
 12: 'Edmund K.',
 13: 'Tsitsipas S.',
 14: 'Medvedev D.',
 15: 'Raonic M.',
 16: 'Cecchinato M.',
 17: 'Schwartzman D.',
 18: 'Basilashvili N.',
 19: 'Dimitrov G.',
 20: 'Goffin D.',
 21: 'Agut R.',
 22: 'Busta P.',
 23: 'Hyeon C.',
 24: 'Shapovalov D.',
 25: 'Verdasco F.',
 26: 'Minaur A.',
 27: 'Pouille L.',
 28: 'Simon G.',
 29: 'Monfils G.',
 30: 'Johnson S.',
 31: 'Kohlschreiber P.'}

In [327]:
indexNames = imd[(~imd['Winner'].isin(x.values())) | (~imd['Loser'].isin(x.values())) | (imd['Surface'] != 'Hard')].index
imd.drop(indexNames, inplace = True)
imd = imd.reset_index(drop=True)

In [328]:
print ("Dataframe shape is:", imd.shape)
num_matches, num_columns = imd.shape
print ("The number of matches is:", num_matches)
print(imd)

Dataframe shape is: (762, 6)
The number of matches is: 762
Attributes Surface          Round        Winner            Loser  WRank  LRank
0             Hard  Quarterfinals  Nishikori K.      Dimitrov G.    9.0   19.0
1             Hard  Quarterfinals   Medvedev D.        Raonic M.   16.0   18.0
2             Hard      The Final  Nishikori K.      Medvedev D.    9.0   16.0
3             Hard      1st Round     Edmund K.    Shapovalov D.   50.0   51.0
4             Hard  Quarterfinals   Dimitrov G.        Edmund K.    3.0   50.0
5             Hard      1st Round   Dimitrov G.       Johnson S.   17.0   33.0
6             Hard      1st Round    Pouille L.         Simon G.   15.0   25.0
7             Hard      2nd Round     Edmund K.       Pouille L.   45.0   15.0
8             Hard      2nd Round     Raonic M.   Schwartzman D.    3.0   52.0
9             Hard  Quarterfinals   Dimitrov G.         Thiem D.   17.0    8.0
10            Hard  Quarterfinals     Raonic M.         Nadal R.    3.0 

In [329]:
# A questo punto posso cancellare la superfice visto che già ho filtrato i dati solo per la superfice Hard
imd = imd.drop('Surface', axis = 1)

In [330]:
LP = imd['Loser'].value_counts()
WP = imd['Winner'].value_counts() 
P  = imd['Winner'].value_counts() - imd['Loser'].value_counts()
nuva_tabella = pd.concat([WP, LP, P], axis=1)
nuva_tabella = nuva_tabella.rename(index={1: "W", 2: "L", 3: "W-L"})

In [331]:
nuva_tabella.columns = ["W","L","W-L"]

In [332]:
nuva_tabella.index.name   = "Player"
nuva_tabella.columns.name = "Attributes"

In [333]:
nuva_tabella.head()

Attributes,W,L,W-L
Player,,,
Anderson K.,35,41,-6
Basilashvili N.,5,20,-15
Cecchinato M.,1,3,-2
Dimitrov G.,38,39,-1
Djokovic N.,108,18,90


In [334]:
imd.head()

Attributes,Round,Winner,Loser,WRank,LRank
0,Quarterfinals,Nishikori K.,Dimitrov G.,9.0,19.0
1,Quarterfinals,Medvedev D.,Raonic M.,16.0,18.0
2,The Final,Nishikori K.,Medvedev D.,9.0,16.0
3,1st Round,Edmund K.,Shapovalov D.,50.0,51.0
4,Quarterfinals,Dimitrov G.,Edmund K.,3.0,50.0


In [359]:
df2020 = imd
df2020 = df2020.rename({'Winner': 'Player_1', 'Loser': 'Player_2'}, axis='columns')
df2020["Winner"] = 1
df2020.head()
print(df2020)

Attributes          Round      Player_1         Player_2  WRank  LRank  Winner
0           Quarterfinals  Nishikori K.      Dimitrov G.    9.0   19.0       1
1           Quarterfinals   Medvedev D.        Raonic M.   16.0   18.0       1
2               The Final  Nishikori K.      Medvedev D.    9.0   16.0       1
3               1st Round     Edmund K.    Shapovalov D.   50.0   51.0       1
4           Quarterfinals   Dimitrov G.        Edmund K.    3.0   50.0       1
5               1st Round   Dimitrov G.       Johnson S.   17.0   33.0       1
6               1st Round    Pouille L.         Simon G.   15.0   25.0       1
7               2nd Round     Edmund K.       Pouille L.   45.0   15.0       1
8               2nd Round     Raonic M.   Schwartzman D.    3.0   52.0       1
9           Quarterfinals   Dimitrov G.         Thiem D.   17.0    8.0       1
10          Quarterfinals     Raonic M.         Nadal R.    3.0    9.0       1
11             Semifinals   Dimitrov G.        Raoni

In [360]:
final = pd.get_dummies(df2020, prefix = ['Player_1','Player_2'], columns = ['Player_1','Player_2'])
final = final.drop("Round", axis = 1)

In [362]:
df2020['Player_1'] = df2020['Player_1'].map({'Djokovic N.' : 1025654400, 'Nadal R.' : 1056153600,
                'Federer R.' : 1208217600, 'Zverev A.' : 712800000,
                'Anderson K.' : 1057536000,'Čilić M.' : 982886400,
                'Thiem D.' : 827280000,'Nishikori K.' : 943401600,
                'Isner J.' : 1090972800,'Khachanov K.' : 741657600,
                'Ćorić B.' : 726364800,
                'Fognini F' : 1025481600,'Edmund K.' : 784771200,
                'Tsitsipas S.' : 671414400,'Medvedev D.' : 750297600,
                'Raonic M.' : 912038400,
                'Cecchinato M.' : 856483200,'Schwartzman D.' : 860371200,
                'Basilashvili N.' : 875491200,
                'Dimitrov G.' : 899942400,' David Goffin' : 913766400,
                'Agut R.' : 997315200,
                'Busta P.' : 895017600,'Hyeon C.' : 741830400,
                'Shapovalov D.' : 650160000,'Verdasco F.' : 1136592000,
                'Minaur A.' : 655084800,
                'Pouille L' : 812332800,'Simon G.' : 1101340800,
                'Monfils G.' : 1048377600,
                'Johnson S.' : 943833600,'Kohlschreiber P.' : 1139184000})
df2020['Player_2'] = df2020['Player_2'].map({'Djokovic N.' : 1025654400, 'Nadal R.' : 1056153600,
                'Federer R.' : 1208217600, 'Zverev A.' : 712800000,
                'Anderson K.' : 1057536000,'Čilić M.' : 982886400,
                'Thiem D.' : 827280000,'Nishikori K.' : 943401600,
                'Isner J.' : 1090972800,'Khachanov K.' : 741657600,
                'Ćorić B.' : 726364800,
                'Fognini F' : 1025481600,'Edmund K.' : 784771200,
                'Tsitsipas S.' : 671414400,'Medvedev D.' : 750297600,
                'Raonic M.' : 912038400,
                'Cecchinato M.' : 856483200,'Schwartzman D.' : 860371200,
                'Basilashvili N.' : 875491200,
                'Dimitrov G.' : 899942400,' David Goffin' : 913766400,
                'Agut R.' : 997315200,
                'Busta P.' : 895017600,'Hyeon C.' : 741830400,
                'Shapovalov D.' : 650160000,'Verdasco F.' : 1136592000,
                'Minaur A.' : 655084800,
                'Pouille L' : 812332800,'Simon G.' : 1101340800,
                'Monfils G.' : 1048377600,
                'Johnson S.' : 943833600,'Kohlschreiber P.' : 1139184000})
df2020['Round']=df2020['Round'].map({'1st Round' : 1,'2nd Round' : 2,'3rd Round' : 3,'4th Round' : 4,
                                      'Quarterfinals' : 5,'Round Robin' : 6,'Semifinals' : 7,'The Final' : 8})

In [363]:
df2020

Attributes,Round,Player_1,Player_2,WRank,LRank,Winner
0,5,9.434016e+08,8.999424e+08,9.0,19.0,1
1,5,7.502976e+08,9.120384e+08,16.0,18.0,1
2,8,9.434016e+08,7.502976e+08,9.0,16.0,1
3,1,7.847712e+08,6.501600e+08,50.0,51.0,1
4,5,8.999424e+08,7.847712e+08,3.0,50.0,1
5,1,8.999424e+08,9.438336e+08,17.0,33.0,1
6,1,NaN,1.101341e+09,15.0,25.0,1
7,2,7.847712e+08,NaN,45.0,15.0,1
8,2,9.120384e+08,8.603712e+08,3.0,52.0,1
9,5,8.999424e+08,8.272800e+08,17.0,8.0,1


In [368]:
df2020=df2020.fillna(1)
print(df2020)

Attributes  Round      Player_1      Player_2  WRank  LRank  Winner
0               5  9.434016e+08  8.999424e+08    9.0   19.0       1
1               5  7.502976e+08  9.120384e+08   16.0   18.0       1
2               8  9.434016e+08  7.502976e+08    9.0   16.0       1
3               1  7.847712e+08  6.501600e+08   50.0   51.0       1
4               5  8.999424e+08  7.847712e+08    3.0   50.0       1
5               1  8.999424e+08  9.438336e+08   17.0   33.0       1
6               1  1.000000e+00  1.101341e+09   15.0   25.0       1
7               2  7.847712e+08  1.000000e+00   45.0   15.0       1
8               2  9.120384e+08  8.603712e+08    3.0   52.0       1
9               5  8.999424e+08  8.272800e+08   17.0    8.0       1
10              5  9.120384e+08  1.056154e+09    3.0    9.0       1
11              7  8.999424e+08  9.120384e+08   17.0    3.0       1
12              8  8.999424e+08  9.434016e+08   17.0    5.0       1
13              1  8.999424e+08  1.101341e+09   

In [369]:
X = final.drop(['WRank'], axis=1)

In [370]:
y = final['WRank']

In [371]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [372]:
final.head()

,WRank,LRank,Winner,Player_1_Anderson K.,Player_1_Basilashvili N.,Player_1_Cecchinato M.,Player_1_Dimitrov G.,Player_1_Djokovic N.,Player_1_Edmund K.,Player_1_Federer R.,...,Player_2_Nishikori K.,Player_2_Pouille L.,Player_2_Raonic M.,Player_2_Schwartzman D.,Player_2_Shapovalov D.,Player_2_Simon G.,Player_2_Thiem D.,Player_2_Tsitsipas S.,Player_2_Verdasco F.,Player_2_Zverev A.
0,9.0,19.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,16.0,18.0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,9.0,16.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,50.0,51.0,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,3.0,50.0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [373]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
score = logreg.score(X_train, y_train)
score2 = logreg.score(X_test, y_test)

In [374]:
print(score)

0.474671669793621


In [375]:
print(score2)

0.20087336244541484


In [378]:
X = df2020.iloc[:, :-1].values
y = df2020.iloc[:, 5].values

In [379]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
print(y_pred)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1]
